# Higgs Challenge
2014年に行われたコンテストを題材に、深層学習を実践してみましょう。

以下は関連するwebページです。適宜参照してください。
* Kaggleのページ: https://www.kaggle.com/c/higgs-boson
* コンテスト結果のサマリーペーパー: http://proceedings.mlr.press/v42/cowa14.pdf
* Datasetの説明ページ: http://opendata.cern.ch/record/328
* Starting Kit: https://higgsml.lal.in2p3.fr/software/starting-kit/


In [ ]:
# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env TF_NUM_INTEROP_THREADS=1
%env TF_NUM_INTRAOP_THREADS=1

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## データセットのダウンロード・読み込み
提供しているバーチャルマシンを使用している場合は`/data/staff/deeplearning`にデータセットがおいてあるので、これを使ってください。

その他の環境を使用している場合は各自データをダウンロード・解凍をしてください。
```bash
$ wget http://opendata.cern.ch/record/328/files/atlas-higgs-challenge-2014-v2.csv.gz
$ gunzip atlas-higgs-challenge-2014-v2.csv.gz
```

CSVデータの読み込み・処理には、今回はpandasというライブラリを使ってみます。
`read_csv`の引数は、CSVデータがある場所に適宜書き換えてください。

In [ ]:
# csvファイルの読み込み
import pandas as pd
df = pd.read_csv("/data/staff/deeplearning/atlas-higgs-challenge-2014-v2.csv")

http://opendata.cern.ch/record/328 で説明されているように、多くの変数が定義されています。それぞれの変数の定義は各自確認してください。

In [ ]:
# 変数の各種統計量の表示
df.describe()

## 使う変数の選別
いろいろな変数がありますが、ここでは使用できる全ての変数を使ってみることにします。

In [ ]:
# 全ての変数を使う例
X = df[[
    'DER_mass_MMC',
    'DER_mass_transverse_met_lep',
    'DER_mass_vis',
    'DER_pt_h',
    'DER_deltaeta_jet_jet',
    'DER_mass_jet_jet',
    'DER_prodeta_jet_jet',
    'DER_deltar_tau_lep',
    'DER_pt_tot',
    'DER_sum_pt',
    'DER_pt_ratio_lep_tau',
    'DER_met_phi_centrality',
    'DER_lep_eta_centrality',
    'PRI_tau_pt',
    'PRI_tau_eta',
    'PRI_tau_phi',
    'PRI_lep_pt',
    'PRI_lep_eta',
    'PRI_lep_phi',
    'PRI_met',
    'PRI_met_phi',
    'PRI_met_sumet',
    'PRI_jet_num',
    'PRI_jet_leading_pt',
    'PRI_jet_leading_eta',
    'PRI_jet_leading_phi',
    'PRI_jet_subleading_pt',
    'PRI_jet_subleading_eta',
    'PRI_jet_subleading_phi',
    'PRI_jet_all_pt'
]]

# 一部だけの変数を使う例
# X = df[['DER_mass_MMC', 'DER_mass_transverse_met_lep', 'DER_mass_vis']]

In [ ]:
# 目標変数とweightの指定
Y = df['Label']
W = df['KaggleWeight']

Label はそのイベントがシグナル、すなわち$H\rightarrow \tau\tau $事象か、バックグラウンドかを表すラベルです。(シグナルは`s`, バックグラウンドは`b`)

KaggleWeightはKaggleでのコンテストで使われたイベントウェイトです。この値が大きいほど、重要なイベントだと思ってください。モデルの評価時には、このweightを使って性能評価をします。

### 前処理(Preprocessing)
必要に応じて、前処理を行ってください。

In [ ]:
# -999 が入っているところを 0 に変換
X = X.replace(-999., 0.)

扱いやすように、それぞれの変数をnumpy形式に変換しておきます。

また、ラベルの`s`,`b`,を数値(`s`=1, `b`=0)に変換します。

In [ ]:
# X を numpy.array 形式に変換します。
X = X.values

# Y を s/b から 1/0に変換します。
from sklearn.preprocessing import LabelEncoder
Y = LabelEncoder().fit_transform(Y)

# W を numpy.array 形式に変換します。
W = W.values

### データの分割

トレーニング用データセットと評価用データセットに分けます。

トレーニングには、`KaggleSet`の値が`t`(training)のものを使ってください。
モデルの評価には、`KaggleSet`の値が`v`(private leaderboard)のものを使ってください。

In [ ]:
X_train = X[df['KaggleSet'] == 't']
Y_train = Y[df['KaggleSet'] == 't']
W_train = W[df['KaggleSet'] == 't']

X_test = X[df['KaggleSet'] == 'v']
Y_test = Y[df['KaggleSet'] == 'v']
W_test = W[df['KaggleSet'] == 'v']

## 深層学習モデルの作成とトレーニング

適当なニューラルネットワークを作成、学習させてみます。

In [ ]:
# DNN の学習
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Reshape, Dense

# モデルの定義
model = Sequential([
    Dense(128, activation='relu', input_shape=X.shape[1:]),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(
    x=X_train,
    y=Y_train,
    batch_size=1000,
    epochs=10
)

## 評価指標の計算

このコンテストでは、AMS(approximate median significance)という指標が使われました。AMSは以下のように定義されています。
$$
\text{AMS} = \sqrt{2 \left( ( s + b + 10 ) \log \left( 1 + \frac{s}{b + 10} \right) - s \right)}
$$
(詳細はhttp://opendata.cern.ch/record/328 を参照してください。)
ここで、`s`はシグナルの数、`b`はバックグラウンドの数です。

この指標を最大にするようなモデルを作成してください。

この指標を計算する関数を以下に用意しました。

In [ ]:
def _ams(s, b):
    import math
    br = 10.0
    return math.sqrt(2 * ( (s + b + br) * math.log(1.0 + s / (b + br)) - s ))

def ams(y_true, y_pred, w, thr):
    # yのshapeを1次元にします。(例: (100, 1) -> (100,))
    y_true = y_true.ravel()
    y_pred = y_pred.ravel()

    # NN output が thr 以上(selectionをpassした)イベントの内、真のラベルが1(シグナル)のもの
    s = w[np.logical_and(y_true == 1, y_pred >= thr)].sum()
    # NN output が thr 以上(selectionをpassした)イベントの内、真のラベルが0(バックグラウンド)のもの
    b = w[np.logical_and(y_true == 0, y_pred >= thr)].sum()

    return _ams(s, b)

def get_best_thr(y_true, y_pred, w):
    import numpy as np
    thresholds = np.linspace(0, 1, 1000)
    ams_l = [ams(y_true, y_pred, w, thr) for thr in thresholds]

    # 閾値をスキャンさせた中で、最もAMSが高かったものを返します。 
    bestIndex = ams_l.index(max(ams_l))
    return thresholds[bestIndex]


上でトレーニングしたモデルでAMSを計算してみます。

In [ ]:
# トレーニングサンプルを使ってNN outputに対する最適な閾値を計算します。
Y_pred = model.predict(X_train)
thr = get_best_thr(Y_train, Y_pred, W_train)
print(f"thr = {thr}")

# 評価用サンプルで AMS を計算します。
Y_pred = model.predict(X_test)
print(f"AMS (test) = {ams(Y_test, Y_pred, W_test, thr)}")


コンテストでは、一位の人のスコアは 3.8 でした。
これを超えることはできるでしょうか？

ぜひいろいろ試してみてください。その際、操作の意味を考えて試行錯誤することを推奨します。

Scoreの参考値
- 1st place: 3.80581
- MultiBoost: 3.40487
- simple TMVA boosted trees: 3.19956
- Naive Bayes starting kit: 2.06020
- Simple window: 1.53518
- Random submission: 0.58647